# Auswertung: Polardiagramme und Drift im Vergleich

## Vorbereitung

Hier importieren wir einige Programpakete (z.B. zum Darstellen der Daten), setzen die Pfade, wo die Dateien zu finden sind und laden
die Daten von 2006, 2008, 2011 und 2012. Beim Einlesen wird bereits automatisch der Korrkturfaktor für die Geschwindigkeit des Log (BSP), den wir zuvor von Hand bestimmt haben, angewandt.

In [1]:
import os
import sys

import numpy as np
import itertools
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# on my computer NX2 is not installed, I just import directly from the development directory
sys.path.append(os.path.abspath('../NX2'))
import NX2
import NX2.polar
from NX2.polar import my_polar
import NX2.math
sys.path.append(os.path.abspath('.'))
import utils

datapath = '/home/moritz/Dropbox/NX2/'
datapath = '/melkor/d1/guenther/Dropbox/NX2/'
plotpath = os.path.join(datapath, '2013/')
ENTWURF = False

ImportError: cannot import name my_polar

In [ ]:
d06 = utils.read_06(datapath)
d08 = utils.read_08(datapath)
d11 = utils.read_11(datapath)
d12 = utils.read_12(datapath)
d1112 = utils.read_11(datapath)
d1112.append(d12)

### Kontolle der BSP / SOG Korrektur

In [ ]:
fit06 = d06.fit_BSP_corr()
fit06[0].pprint()

In [ ]:
fit08 = d08.fit_BSP_corr()
fit08[0].pprint()

In [ ]:
fit11 = d11.fit_BSP_corr()
fit11[0].pprint()

In [ ]:
fit12 = d12.fit_BSP_corr()
fit12[0].pprint()

In [ ]:
fig = plt.figure(figsize = (20,4))
for i, dat, fit, year in zip((1,2,3,4), (d06, d08, d11, d12), (fit06, fit08, fit11, fit12), (2006, 2008, 2011, 2012)):
    ax = fig.add_subplot(1,4,i)
    ax.plot(dat.SOG[~fit[1]], dat.BSP[~fit[1]],'y.', label = 'ignorierte Daten')
    ax.plot(dat.SOG[fit[1]], dat.BSP[fit[1]],'k.', label = 'Datenpunkte')
    xlim = ax.get_xlim()
    ax.plot(xlim, fit[0].beta * xlim, label = 'Fit')
    ax.plot(xlim, 1. * np.array(xlim), 'r', label = '1.0')
    ax.set_xlabel('SOG - Geschwindigkeit vom GPS')
    ax.set_ylabel('BSP - Geschwindigkeit vom Log')
    ax.legend(loc = 'upper left')
    ax.set_title('Daten von {0}'.format(year))
plt.show()

Hier vergleichen wir die SOG mit der BSP.
In dieser Grafik sieht man alle Datenpunkte, die wir für je ein Jahr geladen haben (wir berücksichtigen hier nur die letzten beiden Tage der Messfahrten in 2006, weil wir nur an diesen beiden Tagen gesegelt sind). Die gelbe Wolke zeigt alle Daten, die schwarzen Punkte nur die Daten, die zur Kalibration des Log geeignet sind. Dafür verlangen wir, dass

- die Richtung der Bewegnung über Grund (vom GPS) und die Richtung, in die der Bug zeigt, übereinstimmen, weil es in dieser Position die Drift keine Rolle spielt.
- die Geschwindigkeit > 0.4 kn ist, weil bei Geschwindigkeiten unter dieser Schwelle das Log noch nicht arbeitet.
- die Geschwindigkeit des Schiff sich in diesem Zeitraum nur wenig geändert hat (Aus technischen Gründen hinkt die SOG der BSP immer etwas hinterher. Messungen, die z.B. beim Aufstoppen aufgenommen wurden, sind daher fur die Logkalibration nicht geeignet).

Die blaue Linie zeigt einen Fit durch die schwarzen Punkte. Wenn sie mit der roten Linie übereinstimmt, dann ist keine weitere Korrktur nötig.

Wenn man die Grafiken für die Daten aus jedem einzelnen Jahr anschaut, dann sieht man, dass die Korrekturfaktoren für die BSP ("boat speed" - Geschwindigkeit mit dem Log gemessen) offensichtlich gut gewählt sind und wir ab jetzt die BSP für alle Winkel verwenden konnen, um Polardiagramme zu erzeugen.

## Polardiagramme

### Filtern der Daten

Bevor wir die Daten zu Polardiagrammen zusammenstellen, müssen wir sie filtern, denn nicht jeder Datenpunkt kann dafür genutzt werden. In diesem Abschnitt untersuchen wir, welche Filter sinnvoll sind. Die verschiedenen Filter, die wir auf die Daten anwenden, sollen verschiedene Effekte berücksichtigen: Zuerst beschränken wir uns auf die Messpunkte, die beim Segeln aufgenommen wurden (ohne Hilfe der Ruderer). Dann verlangen wir, dass die Geschwindikgeit sich nur langsam ändert, denn wenn z.B. der Bug in den Wind gedreht wird, dann hat das Schiff zunächst noch eine Geschwindigkeit, die aber schnell abnimmt. Das bedeutet nicht, das wir gegen den Wind segeln können, sondern nur dass die Daten, die direkt nach einer Richtungsänderung, direkt nach dem Hissen des Segels oder in einer Windböhe aufgenommen wurden, herausgefiltert werden müssen. Außerdem ändert sich der Wind sehr viel schneller, als das Schiff seine Geschwindigkeit anpassen kann. Deshalb glätten wir die Windgeschwindigkeit etwas, um diesen Effekt auszugleichen. Dafür wählen wir eine Funktion, die nur vorausgegangene, aber nicht zukünftige Messwerte verwendet; deshalb erscheinen die Maxima im geglätteten Wind immer etwas später. Die nächste Grafik zeigt ein Beispiel dafür.

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.plot(d11['TWS'][3600:3800], label = 'gemessener Wind')
ax.plot(NX2.math.smooth_expdec(d11['TWS'], 10)[3600:3800], label = u'gegl\u00e4tteter Wind')
ax.set_xlabel('Zeit [Sekunden]')
ax.set_ylabel('Windgeschwindigkeit [Knoten]')
ax.legend()
ax.set_ylim([0,10.9])
fig.subplots_adjust(top=0.95, right=0.95, bottom=0.16)
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath, 'windglatt.pdf'))

In der nachsten Grafik schauen wir uns den Winkel zum Wind an. Dabei bedeutet $0^{\circ}$, das die Galeere "im Wind" steht (der Bug in den Wind zeigt), während $180^{\circ}$ meint, das der Wind von achtern kommt und die Galeere "vor dem Wind" segelt. Das Vorzeichen kann dabei positiv oder negativ sein, je nachdem ob der Wind von Steuerbord oder von Backbord kommt. 
Durch diese Skala kann es bei achterlichem Wind zu scheinbaren Spüngen kommen (z. B. um Sekunde 15 herum in der folgenden Grafik), wenn der Wind von $+179^{\circ}$ zu $-179^{\circ}$ wechselt. Da wir in der folgenden Analyse nicht zwischen Backbord und Steuerbord unterscheiden, genügt es, den Betrag des Windwinkels zu betrachten und so das Problem zu vermeiden. Außerdem sieht man in der Grafik, dass die gemessene Windrichtung sehr schnell hin- und herspringt, weil der Windmesser im Wind flattert. Dies ist besonders bei schwachen Wind der Fall und bei Wind vom Bug, wenn das Segel den Windmesser abdeckt. Der Windmesser ist zwar mit einem "twin fin" System ausgestattet, dass ein Hin-und Herflattern vermeiden soll, aber die gemessene Windrichtung schwankt immer noch im Sekundenbereich. Wenn wir dies nicht berücksichtigen, dann werden bei der Konstruktion eines Polardiagrams viele Messwerte bei unpassenden Windwinkeln einsortiert. 
Deshalb glätten wir auch die Windrichtung (z. B. schwankt der Windwinkel in der folgenden Grafik kurz vor Sekunde 150 von fast 0 auf 150, während sich die Geschwindigkeit in dieser kurzen Zeit kaum verändert.).

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax.plot(d11['TWA'][3600:3800], label = 'Windwinkel')
ax.plot(np.abs(d11['TWA'][3600:3800]), label = "Betrag des Winkels")
ax.plot(NX2.math.smooth_expdec(np.abs(d11['TWA'][3600:3800]), 10), label = u"Winkel geglättet")
ax.set_xlabel('Zeit [Sekunden]')
ax.set_ylabel('Winkel zum Wind [Grad]')
ax.legend(loc = 'lower left')

fig.subplots_adjust(top=0.95, right=0.95,left=.2)
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath, 'windwinkelglatt.pdf'), transparent=True)

Um sinnvolle Parameter zum Filtern zu finden und zu zeigen, wie sich die einzelnen Filter auswirken, verwende ich im Folgenden den Datensatz von 2011 als Beispiel.

In [ ]:
sail  = NX2.polar.sail(d11)
con = NX2.polar.near_const(NX2.math.smooth_gauss(d11['BSP'],10), max_diff = 0.007)
TWSs = NX2.math.smooth_expdec(d11['TWS'], 10)
TWAs = NX2.math.smooth_expdec(np.abs(d11['TWA']), 10)
conTWA = NX2.polar.near_const(TWAs, max_diff = 1.5)

Hier schauen wir mal, wie viele Punkte eigentlich noch übrig bleiben, wenn die oben definierten Filter angewendet werden.

In [ ]:
print 'Anzahl Datenpunkte mit Segel und ohne Ruder', sail.sum()
print u'zusätzliche Bedingung: Fast konstante Geschwindigkeit', (sail & con).sum()
print u'zusätzliche Bedingung: Fast konstanter Windwinkel', (sail & con & conTWA).sum()

Alle Punkte mit Segel und ohne Rudern:

In [ ]:
fig = plt.figure(figsize = (10,5))
NX2.polar.grid(d11.TWA[sail], d11.TWS[sail], d11.BSP[sail], np.arange(0.,16.1,4.), np.arange(0.,181., 30.1), fig = fig)

Man sieht schon, dass (außer in der letzten Zeile, wo wir einfach sehr wenige Daten haben) die meisten Verteilungen sehr gut um einen Mittelwert herum liegen. Allerdings sind manche Verteilungen noch so breit, dass man sich fragt, ob wir da vielleicht noch etwas mehr weglassen müssen. In der nächsten Grafik verweden wir nun die geglätteten Werte für Windgeschwindikeit und -winkel. Außerdem filtern wir die Zeiten heraus, bei denen sich die Bootsgeschwindigkeit oder der Windwinkel schnell ändern. Dabei ist es schwer, genau zu definieren, an welcher Stelle man hier abschneiden soll. Ich habe die Werte so gewählt, das nur die wirklich extremen Werte (wenn zum Beispiel eine enge Kurve gesegelt wird) herausgenommen werden. 

In [ ]:
fig = plt.figure(figsize = (10,5))
ind = sail & con & conTWA
NX2.polar.grid(TWAs[ind], TWSs[ind], d11.BSP[ind], np.arange(0.,16.1,4.), np.arange(0.,181., 30.1), fig = fig)

Die Grafik sieht im wesentlichen noch so ähnlich aus wie darüber. Einige der Verteilungen sind etwas schmaler geworden, aber laut dieser Grafik segeln wir immer noch kräftig gegen den Wind. Das haben wir so aber nie direkt beobachtet. Offensichtlich sind die Filter also noch nicht richtig eingestellt. Deshalb schauen wir jetzt mal ein paar der fraglichen Phasen an, um zu verstehen, was da passiert ist.

In [ ]:
indcat = np.zeros(len(d11), dtype = np.bool)
indcat[67500:68500] = True
dat = d11.where(indcat)  #extract data of the time interval in question
ax = dat.plot_speeds()

In [ ]:
print dat.year[0], dat.month[0], dat.day[0]

In [ ]:
fig = plt.figure(figsize = (10,4))
ax = fig.add_subplot(111)
#tax = ax.twinx()
#tax.plot(dat.datetime(), dat.TWA, 'g', label = 'TWA')
ax.plot(dat.datetime(), dat['HDC'], 'k', label = 'Kompasskurs')
ax.plot(dat.datetime(), dat['COG'], 'b', label = u'Kurs über Grund')
ax.plot(dat.datetime(), np.abs(np.mod(np.sign(dat['TWA']) *(dat['HDC'] -dat['COG']), 180.)), 'r', label='Drift Winkel')
ax.legend()
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
ax.set_xlabel("Uhrzeit")
ax.set_ylabel("Winkel")

In diesem Beispiel sind die fraglichen Werte im ersten Drittel und im zweiten Drittel der Grafik zu finden, ca 17:00-17:03 und 17:06-17:10. Auf dem obersten Plot sieht man da nichts Ungewöhnliches, wenn man sich die mit dem Log und die mit dem GPS gemessene Geschwindigkeit anschaut. Auf der nächsten Grafik (direkt über diesem Text) zeigen wir den Kurs über Grund (vom GPS) und den Kompasskurs, sowie die Drift (die Differenz dieser beiden Werte). Zur fraglichen Zeit ist das Schiff deutlich abgedriftet mit einem Driftwinkel um $30^{\circ}$. Zur Kontrolle sind auf der nachsten Grafik die Kurse zum fraglichen Zeitpunkt dargestellt. Dort sieht man deutlich, dass die nominelle Fahrt im Wasser (die Richtung, in die der Bug des Schiffes zeigt - schwarze Pfeile) und die wahre Bewegungsrichtung des Schiffs, die das GPS aufzeichnet (blaue Pfeile) sich unterscheiden. Der nominelle Kurs ist am Wind, aber unter Berücksichtigung der Drift erreichen wir in Wirklichkeit nur "halben Wind" (rechter Winkel zwischen grünen und blauen Pfeilen). 

In [ ]:

n = 60
scale=25
fig=plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax.plot(dat.x, dat.y, 'k')
# overplot path with saling in blue
if 'sailing' in dat.keys():
    # make groups of indices with the sail up
    for sail, ind in itertools.groupby(range(len(dat)), key=lambda a: dat.sailing[a]):
        if sail == 1:
            index = list(ind)
            ax.plot(dat.x[index], dat.y[index], 'b')
wind_v = dat.TWS
wind_ang = dat.TWA + dat.HDC + 180.
quiver_wind = ax.quiver(dat.x[::n], dat.y[::n], dat.TWS[::n]*np.sin(wind_ang[::n]/180.*np.pi), dat.TWS[::n]*np.cos(wind_ang[::n]/180.*np.pi), scale=scale, color='g')
quiver_bsp = ax.quiver(dat.x[::n], dat.y[::n], dat.BSP[::n]*np.sin(dat.HDC[::n]/180.*np.pi), dat.BSP[::n]*np.cos(dat.HDC[::n]/180.*np.pi), scale=scale)
quiver_sog = ax.quiver(dat.x[::n], dat.y[::n], dat.SOG[::n]*np.sin(dat.COG[::n]/180.*np.pi), dat.SOG[::n]*np.cos(dat.COG[::n]/180.*np.pi), scale=scale, color='b')

qk_scale = scale/20.
qk_wind = ax.quiverkey(quiver_wind, .1, 0.95, qk_scale, 'Wind', labelpos='E')
qk_bsp = ax.quiverkey(quiver_bsp, .1, 0.9, qk_scale, 'Fahrt im Wasser', labelpos='E')
qk_sog = ax.quiverkey(quiver_sog, .1, 0.85, qk_scale, u'Fahrt über Grund', labelpos='E')
ax.set_xlabel('West - Ost [Meter]')
ax.set_ylabel(u'Süd - Nord [Meter]')

fig.axes[0].set_aspect("equal")
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')

fig.savefig(os.path.join(plotpath, 'plotcoursehochamwind.pdf'), transparent=True)

Möglicherweise hat der Steuermann hier den Kurs nicht optimal eingestellt und ein Teil der Drift könnte verhindert werden, wenn der Bug des Schiffs nicht in den Wind gestellt wird. Wenn der Bug in den Wind zeigt, dann wird die Drift größer (siehe nächste Grafik, mehr Details unten im Kapitel Drift) ohne dass das Schiff tatsächlich in den Wind navigieren könnte.  Im Folgenden werden wir die wahre Bewegungsrichtung im Wasser betrachten, nicht die Richtung, in die der Bug zeigt. Moderne Schiffe haben normalerweise einen Kiel und die Abdrift ist klein. Auch werden sie bei solchen Tests in der Regel "ideal" gesegelt, also so, dass die größtmögliche Geschwindigkeit erreicht wird. In den Versuchen zu römischen Kriegsschiffen ist das nicht gegeben, weil wir vom ersten Fahrtag an Messwerte aufgezeichnet haben und die verschiedenen Schiffsführer zunächst nicht mit den Besonderheiten dieser Schiffe vertraut waren (da fast 2000 Jahre vergangen sind, seit die Römer diese Schiffe gesegelt haben).

In [ ]:
fig = plt.figure(figsize = (10,5))
ind = sail & con & conTWA
TWAsdrift = np.abs(TWAs)+np.abs(NX2.math.bearingdiff180(d11.HDC, d11.COG))
NX2.polar.grid(TWAsdrift[ind], TWSs[ind], d11.BSP[ind], np.arange(0.,16.1,4.), np.arange(0.,181., 30.1), fig = fig)

**OK, es sind immer noch einige Punkte übrig, dei bei kleinen Winkeln und positiven Geschwindikeiten liegen, aber nicht mehr viele.** Das mache ich iterativ so lange, bis alles, das ich nicht glaube, weggefiltert ist. Dabei habe ich folgende Punkte erkannt und gelöst:

  - Am Anfang habe ich nur für das Segeln die ersten und letzten ca. 1.5 min ignoriert, weil in dieser Zeit ja das Segel erst mal gesetzt und ausgerichtet werden muss (bzw. quer gestellt und eingeholt). Ich habe ausserdem festgestellt, dass viele der fraglischen Bereiche direkt vor oder hinter einem Ruderabschnitt liegen. Hier können wir nicht auschließen, das da vielleicht schon ein "Halt Wasser" gegeben wurde oder die Ruderer ihrer Rimen ins Wasser getaucht haben, um sie besser in die Dollen einhängen zu können. Dashalb filtern wir nun auch Bereiche kurz vor oder nach dem Rudern (ca 1.5 min) heraus.
  - Ich haben den Filter, der prüft, ob die Schifftgeschwindigkeit sich nicht schnell ändert verschärft. Ein guter Wert, um wirklich alle fraglichen Bereuche herauszufiltern, ist eine maximale Änderung von 0.005 pro Sekunde, aber bei einer solchen Bedingung werden auch viele brauchbare Werte verworfen. Als Kompromiss schlage ich jetzt 0.007 als Grenzwert vor.

Wie man oben sieht, bleiben aber selbst im Bereich 30-60 Grad noch einige Punkte übrig. Ich gebe zu bedenken, dass diese alle zu sehr niedrigen BSP gehören, also verträglich mit der Beobachtung "kein Vortrieb" sind. Unten suche ich noch mal die fraglichen Bereiche heraus.

In [ ]:
print (ind & (TWAsdrift < 60) & (TWAsdrift > 30) & (d11.BSP >0.5)).nonzero()
i = 67777 #make -270 be plotted as 90
i = 98650 #
i = 43100
print d11[i]
print d11.names 
print TWAs[i], TWAsdrift[i]

Nun können wir iterativ uns anschauen, welche Datenpunkte nicht herausgefiltert sind, die eine signifikante Geschwindigkeit nach vorn zeigen.

In [ ]:
indcat = np.zeros(len(d11), dtype = np.bool)
indcat[43000:43500] = True
dat = d11.where(indcat)  #extract data of the time interval in question
ax = dat.plot_speeds()

In [ ]:
fig = dat.plot_course(n=30, scale = 20)

In [ ]:
fig = plt.figure(figsize = (10,4))
ax = fig.add_subplot(111)
#tax = ax.twinx()
#tax.plot(dat.datetime(), dat.TWA, 'g', label = 'TWA')
ax.plot(dat.datetime(), dat['HDC'], 'k', label = 'Kompasskurs')
ax.plot(dat.datetime(), dat['COG'], 'b', label = u'Kurs über Grund')
ax.plot(dat.datetime(), np.abs(np.mod(np.sign(dat['TWA']) *(dat['HDC'] -dat['COG']), 180.)), 'r', label='Drift Winkel')
ax.legend()
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M', tz=None))
ax.set_xlabel("Uhrzeit")
ax.set_ylabel("Winkel")

Wenn ich mir das anschaue (der fragliche Punkt ist um 19:03:24 Uhr), dann ändert sich die Geschwindigkeit hier schnell, aber gerade zwischen 19:03 und 19:04 gibt es noch mal ein lokales Maximum, wo dann die lokare Änderung null ist. Meiner Meinung nach ist das ein pathologischer Fall und ich habe keine gute Idee, wie man solche Punkte auch noch erwischt, ohne zu viele brauchbare Punkte mit zu verwerfen.

### Vergleich der Jahre 2006, 2008, 2011 und 2012

Hier zeige ich einen Überblick von Polardiagrammen. Fur jeden Datensatz erzeuge ich die gleichen Diagramme. Die Daten, die in diese Polardiagramme hineingehen, sind wie oben beschrieben gefiltert.

1. Das linke Diagram benutzt in jedem Bin den Median, da der von den nicht gefilterten Ausreissern nicht so stark beeinflusst wird.
2. Das mittelere Diagram bildet den Mittelwert.
3. Das rechte Diagram zeigt den Logarithmus zur Basis 10 der Anzahl der Werte in jedem Bin.

In [ ]:
anglebins=np.arange(0, 181., 15.01)
speedbins=np.arange(1.5,16,3.)

polar06 = my_polar(d06, anglebins=anglebins, speedbins=speedbins)
polar08 = my_polar(d08, anglebins=anglebins, speedbins=speedbins)
polar11 = my_polar(d11, anglebins=anglebins, speedbins=speedbins)
polar12 = my_polar(d12, anglebins=anglebins, speedbins=speedbins)

In [ ]:
def polar3(data, drift=False, anglebins=np.arange(0, 181., 15.01), speedbins=np.arange(1.5,16,3.)):
   
    fig = plt.figure(figsize = (13,7))
    maxr = 5 if not drift else 50
    aux1, ax1 = NX2.polar.setup_plot(fig, 131, maxr=maxr)
    aux2, ax2 = NX2.polar.setup_plot(fig, 132, maxr=maxr)
    aux3, ax3 = NX2.polar.setup_plot(fig, 133)
    colors = ['r','g','b','k','y','c','orange','r']
    for aux, fct in zip([aux1, aux2, aux3], [np.median, np.mean, lambda x: np.log10(len(x))]): 
        poldat = my_polar(data, drift=drift, fct=fct, speedbins=speedbins, anglebins=anglebins)
        temp = NX2.polar.plot(aux, poldat, speedbins, anglebins, color = colors)
    ax1.set_title('Polardiagram - Median')
    ax2.set_title('Polardiagram - Mean')
    ax3.set_title ('Log10(Anzahl Datenpunkte per bin)')
    ax3.axis["left"].label.set_text(r"log10(Anzahl Datenpunkte)")
    if drift:
        for ax in [ax1,ax2]:
            ax.axis["left"].label.set_text(r"Driftwinkel [$^{\circ}$]")

In [ ]:
polar3(d06)

In [ ]:
polar3(d08)

In [ ]:
polar3(d11)

In [ ]:
polar3(d12)

Fur die Geschwindigkeiten unter 6 kn sehen die Grafiken allesamt relativ gut aus. Bei stärkerem Wind haben wir einfach zu wenig Datenpunkte, um wirklich viel aussagen zu konnen. Bei schwachem Wind konnte man die Unterteilung vermutlich sogar noch etwas feiner machen, allerdings kann man das dann nicht mehr gut mit 2006 vergleichen, wo wir in der Tat nur eine sehr kurze Messzeit hatten.

Zu besseren Vergleichbarkeit werden nun die Polardigramme aller Messjahre für jeweils eine bestimmte Geschwindigkeit übereinander dargestellt.

In [ ]:
#speedbins=np.arange(1.5,16,3)

fig = plt.figure(figsize = (25, 5))
for i, tit in zip(range(len(speedbins)), speedbins[:-1]+np.diff(speedbins)/2.):
    aux, ax = NX2.polar.setup_plot(fig, 170 + i+1)
    for pol, lab in zip([polar06, polar08, polar11, polar12], ['2006', '2008','2011','2012']):
        temp = NX2.polar.plot_half_circle(aux, anglebins[0:-1] + np.diff(anglebins)/2., pol[i+1,1:], lw = 3., label = lab)
    ax.legend(loc = 'upper right')
    ax.set_title('TWS ca. '+ str(tit) + ' kn')

Ich lese hier ab, dass die Lusoria Regina am langsamsten ist. Das kann natürlich verschiedene Gründe haben, einer davon ist, dass wir 2006 noch nicht so gut Segeln konnten und nicht das beste aus dem Schiff herausgeholt haben...
Es scheint auch so zu sein, dass wir 2012 tendenziell etwas schneller waren als 2011. Vielleicht hat sich das das größere Segel ausgewirkt. Andererseits ist der Unterschied nicht besonders gross und ich bin wir nicht sicher, das sich die beiden Verteilungen statistisch signifikant unterscheiden (Das könnte man natürlich mit einem KS-Test prüfen, wenn wir das wollen). Für das Buch würde ich die Plots vielleicht auch doch wieder mit breiteren bins machen.

Was liest du hier noch ab?

Ich habe auch mal die Bücher aufgeschlagen und versucht diese neuen Kurven mit dem zu vergleichen, was schon gedruckt ist. Da scheint es mir (auch wenn wir in IDL etwas andere bins verwendet haben), dass die neue Auswertung immer etwas schneller ist. Woran mag das liegen? Haben wir damals die BSP Kalibration nicht so gut im Griff gehabt? Ich dachte immer, das ware einer der schwachsten Punkte in der Analyse, wahrend hier der Vergleich (s. oben) ja zeigt, das wir dieses Problem nun gelöst haben. Wir finde ich das am besten heraus, wo der Unterschied herkommt?

### Polardiagramme für das Buch

In der Buchveroffentlichung sollen auch Polardigramme gezeigt werden, allerdings nicht alle, die oben gezeight werden, sondern nur eine Auswahl davon (z.B. wollen wir dort den Unterschied zwischen mean und median nicht diskutieren). In den folgenden Zellen werden die Polardigramme erzeugt; sind enthalten keine neuen Informationen, sondern sind nur anders präsentiert. Auch die bins sind etwas anders gewählt.

In [ ]:
anglebinsbook = np.arange(0, 181., 15.0001)
speedbinsbook = np.arange(1,12,2)

In [ ]:
fig = plt.figure(figsize = (3,5))
aux, ax = NX2.polar.setup_plot(fig, 111, maxr=5)
colors = ['r','g','b','k','r','c','orange','r']
poldat = my_polar(d11, rawTWA=True, speedbins=speedbinsbook, anglebins=anglebinsbook)
temp = NX2.polar.plot(aux, poldat, speedbinsbook, anglebinsbook, color = colors)
ax.set_title('Polardiagram - 2011')
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath,'polar11.pdf'))

In [ ]:
fig = plt.figure(figsize = (3,5))
aux, ax = NX2.polar.setup_plot(fig, 111, maxr=5)
colors = ['r','g','b','k','r','c','orange','r']
poldat = my_polar(d11, speedbins=speedbinsbook, anglebins=anglebinsbook)
temp = NX2.polar.plot(aux, poldat, speedbinsbook, anglebinsbook, color = colors)
ax.set_title('Polardiagram - 2011\nDrift korrigiert')
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath,'polar11korr.pdf'))

In [ ]:
speedbins2=np.array([3.,6., 9.])

polar06b = my_polar(d06, speedbins=speedbins2, anglebins=anglebinsbook)
polar08b = my_polar(d08, speedbins=speedbins2, anglebins=anglebinsbook)
polar11b = my_polar(d11, speedbins=speedbins2, anglebins=anglebinsbook)
polar12b = my_polar(d12, speedbins=speedbins2, anglebins=anglebinsbook)

fig = plt.figure(figsize = (5.5, 4))
aux1, ax1, grid_helper1 = NX2.polar.setup_plot(fig, 121, maxr=3.5, returnall=True)
aux2, ax2, grid_helper2 = NX2.polar.setup_plot(fig, 122, maxr=3.5, returnall=True)
for pol, lab, col in zip([polar06b, polar08b, polar11b, polar12b], ['Regina 2006', 'Victoria 2008','Rhenana 2011','Rhenana 2012'], ['orange', 'purple', 'fuchsia', 'olive']):
    temp = NX2.polar.plot_half_circle(aux1, anglebinsbook[0:-1] + np.diff(anglebinsbook)/2., pol[1,1:], lw = 3., label = lab, ls='-', color=col, marker='o')
    temp = NX2.polar.plot_half_circle(aux2, anglebinsbook[0:-1] + np.diff(anglebinsbook)/2., pol[2,1:], lw = 3., label = lab, ls='-', color=col, marker='o')

ax2.legend(loc = 'upper right', bbox_to_anchor=(1.3, 1.03))
ax1.set_title('Wind 3-6 Knoten')
ax2.set_title('Wind 6-9 Knoten')
grid_helper1.grid_finder.grid_locator2._nbins=7
grid_helper2.grid_finder.grid_locator2._nbins=7
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath,'polarVergleich.pdf'), transparent=True)

In [ ]:
speedbins2

In [ ]:
anglebinsbook

In [ ]:
polar12b.shape

## Drift

In [ ]:
polar3(d06,drift=True)

In [ ]:
polar3(d08, drift=True)

In [ ]:
polar3(d11,drift=True)

In [ ]:
polar3(d12,drift=True)

In [ ]:
drift06 = my_polar(d06, drift=True)
drift08 = my_polar(d08, drift=True)
drift11 = my_polar(d11, drift=True)
drift12 = my_polar(d12, drift=True)

In [ ]:
fig = plt.figure(figsize = (25, 5))
for i, tit in zip(range(len(speedbins)), speedbins[:-1]+np.diff(speedbins)/2.):
    aux, ax = NX2.polar.setup_plot(fig, 170 + i+1, maxr=50)
    for pol, lab in zip([drift06, drift08, drift11, drift12], ['2006', '2008','2011','2012']):
        temp = NX2.polar.plot_half_circle(aux, anglebins[0:-1] + np.diff(anglebins)/2., pol[i+1,1:], lw = 3., label = lab)
    ax.legend(loc = 'lower left')
    ax.set_title('Wind ca. '+ str(tit) + ' kn')
    ax.axis["left"].label.set_text(r"Driftwinkel [$^{\circ}$]")

In [ ]:
drift06 = my_polar(d06, drift=True, speedbins=speedbins2, anglebins=anglebinsbook)
drift08 = my_polar(d08, drift=True, speedbins=speedbins2, anglebins=anglebinsbook)
drift11 = my_polar(d11, drift=True, speedbins=speedbins2, anglebins=anglebinsbook)
drift12 = my_polar(d12, drift=True, speedbins=speedbins2, anglebins=anglebinsbook)

fig = plt.figure(figsize = (5.5, 4))
aux1, ax1, grid_helper1 = NX2.polar.setup_plot(fig, 121, maxr=50, returnall=True)
aux2, ax2, grid_helper2 = NX2.polar.setup_plot(fig, 122, maxr=50, returnall=True)

for pol, lab, col in zip([drift06, drift08, drift11, drift12], ['Regina 2006', 'Victoria 2008','Rhenana 2011','Rhenana 2012'], ['orange', 'purple', 'fuchsia', 'olive']):
    temp = NX2.polar.plot_half_circle(aux1, anglebinsbook[0:-1] + np.diff(anglebinsbook)/2., pol[1,1:], lw = 3., label = lab, ls='-', color=col, marker='o')
    temp = NX2.polar.plot_half_circle(aux2, anglebinsbook[0:-1] + np.diff(anglebinsbook)/2., pol[2,1:], lw = 3., label = lab, ls='-', color=col, marker='o')

ax1.axis["left"].label.set_text(r"Driftwinkel [Grad]")
ax2.axis["left"].label.set_text(r"Driftwinkel [Grad]")
ax2.legend(loc = 'lower left', bbox_to_anchor=(0.1, -.05))
ax1.set_title('Wind 3-6 Knoten')
ax2.set_title('Wind 6-9 Knoten')
grid_helper1.grid_finder.grid_locator2._nbins=5
grid_helper2.grid_finder.grid_locator2._nbins=5
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath,'driftVergleich.pdf'), transparent=True)

## Vergleich mit Widerstandsmessungen im Tank

In [ ]:
fig = plt.figure(figsize=(4.5,3))
ax = fig.add_subplot(111)
speedbinsmid = speedbins[:-1]+np.diff(speedbins)/2.
# star plot at 0
speedbinsmid = np.hstack([[0], speedbinsmid])
for pol, lab in zip([polar06, polar08, polar11, polar12], ['2006', '2008','2011','2012']):
    # np.hstack([[0], pol[1:-1,-1]]) ignores the first bin pol
    # This bin hald the information for speeds below the lowest boundary in speedbins
    # We want to ignore this nomber and set it to 0, because the log does not record
    # very low speeds, so this bin is not reliable
    ax.plot(np.hstack([[0], pol[1:-1,-1]]), speedbinsmid, label=lab, lw=2., marker='o')
ax.set_xlabel(u'Fahrt über Grund [Knoten]')
ax.set_ylabel('Windgeschwindigkeit [Knoten]')
ax.set_title('Wasserwiderstand')
ax.legend(loc='upper left')
fig.subplots_adjust(bottom=0.15)
if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath,'VergleichTank.pdf'), transparent=True)

Nun vergleichen wir die am Nachbau gemessenen Geschwindigkeiten mit den Widerstandsmessungen am Model im Tank zu vergleichen (siehe Kapitel von XXX in diesem Band). Im Tank werden die Modelle bei definierter Geschwindigkeit bewegt und die dazu notwendige Kraft gemessen. Am Nachbau kennen wir die Kraft, die auf das Schiff wirkt, nicht genau, aber die Kraft, die das Schiff beim Segeln antreibt, ist in etwa proportional zur Windgeschwindigkeit. Die dargestellten Werte sind alle bei Wind von achtern gemessen. In dieser Position ist es leicht, das Segel optimal einzustellen. 
Für alle von Wasser umströmten Objekte gilt, dass die benötigte Kraft exponentiell mit der Geschwindigkeit steigt und in den Messungen der Schiffbauversuchsanstalt in Potsdam wurde dies auch für alle getesteten Modelle römischer Kriegsschiffe bestätigt. In der Grafik ist dieser Zusammenhang nicht gut zu erkennen, weil die gemessenen Geschwindigkeiten relativ klein sind und von den bereits diskutierten Messungenauigkeiten beeinflusst werden. Die Daten der Schiffsbauversuchsanstalt zeigen zun Beispiel, dass für das Schiff vom Typ Oberstimm (2008) nur eine Kraft von 0,22 kN benötigt wird, um 4 kn zu erreichen, während 1 kN für 8,0 kn erforderlich ist. So schnell sind wir aber im Experiment nie gesegelt (der Mast brach, bevor diese Geschwindigkeit erreicht werden konnte). Andererseits stimmen die am Nachbau gemessenen Werte qualitativ gut mit den Widerstandsmessungen am verkleinerten Modell im Tank überein. Bei einer Geschwindigkeit von 3 kn ergibt der Versuch im Wassertank, dass die Lusoria Regina (2006) ca 80% mehr Antriebskraft erfordert als die Victoria (2008). Auch am Nachbau erfordert die Regina durchgängig einen stärkeren Wind als die Victoria, um eine bestimmte Geschwindigkeit über Grund zu erreichen. 

Im Vergleich der Victoria und der Lusoria Rhenana (2011 und 2012) ergeben die Tests im Tank, dass die Victoria eine geringere Kraft erfordert. Am Nachbau hingegen unterscheiden sich diese beiden Schiffe kaum.

### Ein paar Situationen, die wir gern mal von Hand anschauen wollen

In [ ]:
dat = d11.where(d11['day'] == 12)
dat = dat.when((12,45,00),(13,00,00))
filename = '/home/moritz/Dropbox/NX2/2013/'

In [ ]:
ax = dat.plot_speeds()
plt.savefig(plotpath+'ex_speed.png')

In [ ]:
fig = dat.plot_course(n=30, scale = 20)
fig.savefig(plotpath + 'ex_course.png')

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = fig.add_subplot(111)
ax.plot(dat['TWA'], label = 'Windwinkel')
ax.plot(np.abs(dat['TWA']), label = "Betrag des Winkels")
ax.plot(NX2.math.smooth_expdec(np.abs(dat['TWA']), 10), label = u"Winkel geglättet")
ax.set_xlabel('Zeit [Sekunden]')
ax.set_ylabel('Winkel zum Wind [$^{\circ}$]')
ax.legend(loc = 'lower left')
fig.subplots_adjust(top=0.97, right=0.97)
plt.savefig(plotpath + '_wind.png')

In [ ]:
plt.plot(dat['TWS'])
plt.ylabel('TWS')
plt.savefig(plotpath + 'ex_TWS.png')

## Grafiken zur Erklärung des Polardigrams

Um zu erklären, was ein Polardigram ist, brauchen wir ein paar Beispielgrafiken.

In [ ]:
fig = plt.figure(figsize = (3,5))
aux, ax = NX2.polar.setup_plot(fig, 111, maxr=5)
colors = ['r','g','b','k','r','c','orange','r']
#poldat = my_polar(d11, rawTWA=True, speedbins=speedbinsbook, anglebins=anglebinsbook)
#temp = NX2.polar.plot(aux, poldat, speedbinsbook, anglebinsbook, color = colors)
#ax.set_title('Polardiagram')
image = plt.imread('ship2.png')
axicon = fig.add_axes([-0.15,0.15,0.8,0.8], frameon=False)
axicon.imshow(image)
axicon.set_xticks([])
axicon.set_yticks([])

ax.annotate('vor dem\nWind', xy=(.05, .4),  xycoords='axes fraction',
                xytext=(0.05, .1), textcoords='axes fraction', ha='center', va='center',
                size=20,
                bbox=dict(boxstyle="round", fc=(1.0, 0.7, 0.7), ec=(1., .5, .5)),
                arrowprops=dict(arrowstyle="fancy",
                                fc=".3", ec="none",
                                connectionstyle="angle3,angleA=0,angleB=-90"),
                )

ax.annotate('halber\nWind', xy=(.2, .5),  xycoords='axes fraction',
                xytext=(.6,.5), textcoords='axes fraction', va='center',
                size=20,
                bbox=dict(boxstyle="round", fc=(1.0, 0.7, 0.7), ec=(1., .5, .5)),
                arrowprops=dict(arrowstyle="fancy",
                                fc=".3", ec="none",
                                connectionstyle="angle3,angleA=0,angleB=-90"),
                )

ax.annotate('im\nWind', xy=(.05, .65),  xycoords='axes fraction',
                xytext=(.05,.95), textcoords='axes fraction', ha='center', va='center',
                size=20,
                bbox=dict(boxstyle="round", fc=(1.0, 0.7, 0.7), ec=(1., .5, .5)),
                arrowprops=dict(arrowstyle="fancy",
                                fc=".3", ec="none",
                                connectionstyle="angle3,angleA=0,angleB=-90"),
                )

aux.plot(150+20.*np.random.random(50),np.random.normal(3., .5, 50),'b<')
aux.plot(130+20.*np.random.random(50),np.random.normal(2.5, .5, 50),'r<')
aux.plot(110+20.*np.random.random(50),np.random.normal(2., .5, 50),'g<')
aux.plot(np.array([150,130,110])+10,[3,2.5,2],'k',lw=4,marker='o', ms=10, zorder=3)

#if ENTWURF: fig.text(.5,.5, 'ENTWURF', rotation=45,ha='center', va='center', fontsize=35, alpha=.5, color='grey')
fig.savefig(os.path.join(plotpath,'polar_beispiel.pdf'))

## Abspeichern der Ergebisse
Jetzt speichern wir die Polardigramme jeweils als Datei ab.